In [1]:
import secrets
from sympy import mod_inverse
import numpy as np
from math import floor, log
import time

# Physics 566: Final Project - RSA Encyptrion 

Cameron Foltz, 2020

In [2]:
#Primality test

def isPrime(n, iterations):
    """Miller-Rabin Primality test (Probabilistic)
    
    Parameters:
    n: integer to be tested
    interations: how many times the test is performed
    
    output: if Probably Prime --> True
            if Composite --> False
    """
    
    # Check if even or < 1
    if n % 2 == 0 or n <= 1:
        return False 
    if n <= 3:
        return True
    
    # Find largest a such that 2^a divides n-1
    a = 0
    while ((n-1) % 2**a) == 0:
        a += 1
    
    a = a-1 

    wlen = (n-1).bit_length()
    m = (n-1) // (2**a)
    
    # Perform Miller-Rabin
    for k in range(iterations):
        
        #Generate random bits 1 < b < w-1
        b = secrets.randbits(wlen)
        
        if b <= 1 or b >= n-1:
            continue
        
        z = pow(b,m,n)
        
        if z != 1 and z != n-1:
            for j in range(1,a):
                z = pow(z,2,n)
                if z == 1:
                    return False
                if z == n-1:
                    break
            if z != n-1:
                return False
    return True

In [3]:
# Random Prime Number Generation
def primeGeneration(pLength,qLength, iterations):
    """Generates two random prime numbers p,q
    Parameters:
    pLength: Desired Bit Length of p
    qLength: Desire Bit Length of q
    iterations: Number of iterations for primality test
    
    output:
    two prime integers p,q of length pLength,qLength
    """
    
    # Generate p
    prime1 = False
    
    while prime1 == False:
        test1 = secrets.randbits(pLength)
        
        #Make sure bit length is correct
        while test1.bit_length() != pLength:
            test1 = secrets.randbits(pLength)
            
        # Perform prime test
        prime1 = isPrime(test1,iterations)
        
    p = test1
    
    # Generate q
    prime2 = False
    
    while prime2 == False:
        test2 = secrets.randbits(qLength)
        
        #Make sure bit length is correct
        while test2.bit_length() != qLength:
            test2 = secrets.randbits(qLength)
            
        # Perform prime test
        prime2 = isPrime(test2,iterations)
        
    q = test2
    
    return p,q


In [4]:
# Functions to find Phi
def gcd(a,b):
    """Returns greatest common denominator of integers a,b"""
        while b > 0:
            a, b = b, a % b
        return a
    
def LCM(p,q):
    """Returns the LCM of integers p,q"""
    # Return LCM of a,b
    return p*q // gcd(p,q)

In [25]:
def Encrypt(filePath,publicKey):
    """Parameters:
    :filePath: File Path of text-file to encrypt
    :publicKey: public encryption key in form [n,e]
    
    output:
    Encrypted character array
    """
    
    message = open(filePath)
    message = message.read()
    
    print("The message you wish to encrypt:\n\n"+message+"\n")
    def txtConv(string):
        conv_string = []
        for i in string:
            conv_string.append(ord(i))
        return conv_string    

    # Conv each num of message to cipher: c = m^2 % n w/ public key
    def cipher(array):
        cipher = []
        for i in range(len(array)):
            cipher.append(pow(int(array[i]),publicKey[1], publicKey[0]))
        return cipher
    
    cipher = cipher(txtConv(message))
    
    string = ''.join(str(i) for i in cipher)

    print("Encrypted message:\n" + string)
    return cipher

In [6]:
## Decrypt the message  
    
def Decrypt(message, privKey):
    """Parameters:
    :message: Input cipher text in form of an Array
    :privateKey: Key in form [n,d]
    Returns decrypted message in a string using the 
    standard method w/ simplified key"""
    
    decipher = []
    for i in message:
        decipher.append(pow(i, privKey[1], privKey[0]))
    finalMessage = ""
    for j in decipher:
        finalMessage += chr(j)
    print("The Decrypted Message:\n\n"+ finalMessage)
    

In [13]:
%%time
## Key Generation

# Decide on length of prime numbers (# of bits)
pLength = 2048
qLength = 256

# Number of miller-rabin tests to perform
iterations = 56

# Select two prime numbers
p,q = primeGeneration(pLength,qLength,iterations)
n = p*q
phi = LCM(p-1,q-1)

e = 65537 # Industry Standard
d = mod_inverse(e,phi) 

pubKey = [n,e]
privKey = [n,d]

print(f"Public Key: ({n},{e})")
print(f"Private Key: ({n},{d})")

Public Key: (2669004196143155379307878225855836370690005253256865879685843269199871724470011243982077469396357894755025261767124840875445272079950898769955801849877014022816555948316663038043258729044293216644722298349194305974342734838928715142344009837877566632716996106661090864768885822701645684077308389638725991925069640570760013497222449411887098897514560720137903905461488723235958701751290144786181398626429727354203366058312354547182830491452214195450677891417757119816767106106350842256279800984670399618238727886293336688038592864800344721805613270570891990891954379144902026445624836777168187785989712434244545473170708137588891690071693359325263683973300587334124993341167978121348010928142833,65537)
Private Key: (2669004196143155379307878225855836370690005253256865879685843269199871724470011243982077469396357894755025261767124840875445272079950898769955801849877014022816555948316663038043258729044293216644722298349194305974342734838928715142344009837877566632716996106661090864768

In [26]:
%%time
# Encrypt Message with Keys
path = "./Message2.txt"

secret = Encrypt(path,pubKey)

The message you wish to encrypt:

"The only true wisdom is in knowing you know nothing"

-Socrates

Encrypted message:
...
CPU times: user 10.4 ms, sys: 1.66 ms, total: 12.1 ms
Wall time: 15.8 ms


In [28]:
%%time

# Decrypt Message
Decrypt(secret,privKey)

The Decrypted Message:

"The only true wisdom is in knowing you know nothing"

-Socrates
CPU times: user 2.64 s, sys: 6.99 ms, total: 2.64 s
Wall time: 2.65 s
